In [6]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd

In [7]:
db = database('trfc.db')

In [8]:
app, rt = fast_app()

In [9]:
data = {
    'min_season': 1921,
    'max_season': 2024
}

venues = ('H', 'A', 'N')

min_games = 10

In [26]:
focus = 'r.season'
focus = 'r.opposition'
focus = 'm.manager_name'

In [27]:
query = f'''
    SELECT 
            {focus},
            COUNT(*) as P,
            COUNT(CASE WHEN r.goals_for - r.goals_against == 1 THEN 1 END) as gd_plus_one,
            COUNT(CASE WHEN r.goals_for - r.goals_against == 2 THEN 1 END) as gd_plus_two,
            COUNT(CASE WHEN r.goals_for - r.goals_against == 3 THEN 1 END) as gd_plus_three,
            COUNT(CASE WHEN r.goals_for - r.goals_against == 4 THEN 1 END) as gd_plus_four,
            COUNT(CASE WHEN r.goals_for - r.goals_against >= 5 THEN 1 END) as gd_plus_five_plus,
            COUNT(CASE WHEN r.goals_for == r.goals_against THEN 1 END) as gd_zero,
            COUNT(CASE WHEN r.goals_against - r.goals_for == 1 THEN 1 END) as gd_minus_one,
            COUNT(CASE WHEN r.goals_against - r.goals_for == 2 THEN 1 END) as gd_minus_two,
            COUNT(CASE WHEN r.goals_against - r.goals_for == 3 THEN 1 END) as gd_minus_three,
            COUNT(CASE WHEN r.goals_against - r.goals_for == 4 THEN 1 END) as gd_minus_four,
            COUNT(CASE WHEN r.goals_against - r.goals_for >= 5 THEN 1 END) as gd_minus_five_plus
        FROM results r
        LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
            AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
        LEFT JOIN managers m ON mr.manager_id = m.manager_id
        LEFT JOIN season_league_tiers slt ON r.season = slt.season AND r.game_type = 'League'
        WHERE CAST(SUBSTRING(r.season, 1, 4) AS INTEGER) >= {data['min_season']} AND CAST(SUBSTRING(r.season, 1, 4) AS INTEGER) <= {data['max_season']}
            AND r.venue IN ('H', 'A', 'N')
        GROUP BY {focus}
        HAVING COUNT(*) >= {min_games}
        ORDER BY P DESC
'''

In [42]:
df = pd.read_sql(query, db.conn)

df.sort_values(df.columns[0])

df.columns[2:len(df.columns)]

# third column of df
df.columns[2]

'gd_plus_one'